# 5. Fine-Tuning de XLNet

## 5.1. Introduction

Ce notebook se concentre sur le fine-tuning du modèle **XLNet**, un autre modèle Transformer performant, pour notre tâche de classification de sentiments.

### 5.2. Installation et Importations

In [ ]:
!pip install transformers datasets accelerate bitsandbytes evaluate -q

In [ ]:
import torch
import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments

### 5.3. Chargement et Préparation des Données

In [ ]:
data = pd.read_csv('../data/balanced_subset.csv')
sampled_data = data.sample(frac=0.3, random_state=42)
label_map = {"Positive": 2, "Neutral": 1, "Negative": 0}
sampled_data['rating'] = sampled_data['rating'].map(label_map)
sampled_data.dropna(subset=['cleaned_text'], inplace=True)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sampled_data['cleaned_text'].tolist(), 
    sampled_data['rating'].tolist(), 
    test_size=0.2, 
    random_state=42,
    stratify=sampled_data['rating'].tolist()
)

### 5.4. Tokenisation et Création des Datasets

In [ ]:
model_name = "xlnet-base-cased"
tokenizer = XLNetTokenizer.from_pretrained(model_name)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels,
})
val_dataset = Dataset.from_dict({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": val_labels,
})

### 5.5. Entraînement du Modèle

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)
training_args = TrainingArguments(
    output_dir="./results_xlnet",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    logging_dir="./logs_xlnet",
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

### 5.6. Sauvegarde du Modèle

In [ ]:
output_path = "../models/xlnet"
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)
print(f"Modèle XLNet sauvegardé dans {output_path}")